# IMPORTS

In [5]:
from collections import *
from functools import *
from itertools import *
import sys
import re
import math
import string
import bisect
from parse import compile
import matplotlib.pyplot as plt
import numpy as np
import heapq
from matplotlib.path import Path
import ast


# SOLVING

In [161]:
def solve(elements, groups):
    # state = (current group size, current group index, last character)
    bags = Counter()
    if elements[0] == "?" or elements[0] == "#":
        bags[(1, 0)] += 1
    if elements[0] == "?" or elements[0] == ".":
        bags[(0, 0)] += 1
    n = len(elements)
    m = len(groups)
    for i in range(1, n):
        new_bags = Counter()
        for (cur, j), cnt in bags.items():
            if elements[i] == ".":
                if cur == 0:
                    new_bags[(0, j)] += cnt
                elif cur == groups[j]:
                    new_bags[(0, j + 1)] += cnt
            elif elements[i] == "#":
                if j < m and cur < groups[j]:
                    new_bags[(cur + 1, j)] += cnt
            else:
                if cur == 0:
                    new_bags[(0, j)] += cnt
                    if j < m:
                        new_bags[(1, j)] += cnt
                elif cur == groups[j]:
                    new_bags[(0, j + 1)] += cnt
                elif cur > 0:
                    new_bags[(cur + 1, j)] += cnt
        bags = new_bags
    return sum(cnt for (_, j), cnt in bags.items() if j == m)
def parts(sz):
    with open("big.txt", "r") as f:
        data = f.read().splitlines()
        res = 0
        for line in data:
            first, second = line.split()
            start = list(map(int, second.split(",")))
            elements = "?".join([first for _ in range(sz)]) + "."
            groups = []
            for _ in range(sz):
                groups.extend(start)
            res += solve(elements, groups)
        return res
print("part 1:", parts(1))
print("part 2:", parts(5))

part 1: 7674
part 2: 4443895258186


In [43]:
2**19

524288

In [ ]:
turn_on = compile("turn on {:d},{:d} through {:d},{:d}")
turn_off = compile("turn off {:d},{:d} through {:d},{:d}")
toggle = compile("toggle {:d},{:d} through {:d},{:d}")
with open('input.txt', 'r') as f:
    data = f.read().splitlines()
    lights = [[0] * 1000 for _ in range(1000)]
    for line in data:
        to, toff, tog = turn_on.parse(line), turn_off.parse(line), toggle.parse(line)
        if to is not None:
            r1, c1, r2, c2 = to.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] += 1
        elif toff is not None:
            r1, c1, r2, c2 = toff.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] = max(0, lights[r][c] - 1)
        else:
            r1, c1, r2, c2 = tog.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] += 2
    print(sum(map(sum, lights)))